# Разработка модели для поиска музейных экспонатов


**Задача** - разработать модель, которая будет по текстовому описанию музейного экспоната определять, какая фотография из базы данных ему соответствует, что в дальнейшем позволит создавать выборки предметов по более гибким параметрам поиска.

**Метрика** - R2

**Условия**:
Для того, чтобы метрика отработала корректно, значения в присланных
ответах должны:
1. Не повторяться
2. Не включать в себя индексы изображений из папки train


## Импорты

In [725]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from os import listdir 
import re
import string

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import ngrams
import pymorphy2
import re



nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))

from lightgbm import LGBMRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from scipy.sparse import hstack
from catboost import CatBoostRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anastasia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anastasia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Загрузка данных

In [726]:
df_train = pd.read_csv("train.csv")

In [727]:
df_test = pd.read_csv("test.csv")

In [728]:
print("Обучающей выборки " ,len(listdir("train")))

Обучающей выборки  2098


In [729]:
print("Тестовой выборки " ,len(listdir("test")))

Тестовой выборки  900


## Первичный анализ датасета

In [730]:
df_train.head(10)

,id,description,object_img
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244
5,2048,Фотография. Дом № 124 по ул. Орджоникидзе в г....,917
6,1256,"Фотонегатив пленочный. Пермский НПК, общий вид...",1069
7,2317,ПЗС. Подвеска шумящая коньковая на цепочке ПК...,2114
8,1562,"Этюд. ""Раздумье"". ПОКМ-19728/20 бумага,акваре...",2555
9,2022,Фотокопия. Карл Витте с семьей. ПОКМ-10959/62...,946


In [731]:
df_test.head(10)

,id,description
0,486,Фотография. Елизавета Алексеевна Юманова. ПКМ...
1,813,Фотография. Заседание комитета комсомола мотор...
2,2980,"Фотография. День ""Саланга"". ПОКМ-18530/638 фо..."
3,13,Фотография. Елизавета Алексеевна Юманова. ПКМ...
4,2467,"Фотография. ""Универмаг Пермь. Оформление демон..."
5,2526,Фотография. Артисты балета на открытии учебног...
6,297,Монета 2 копейки 1800 г. А/РУС-298 ПКМ-21218/9...
7,2981,Фотография Анна Павловна Горюнова. ПКМ-21492/...
8,2055,"Стакан, артиллерийского снаряда. ОР-174 ПОКМ-4..."
9,45,"Этюд. ""Речной берег"". ПКМ-21326/11 масло,карт..."


In [732]:
df_train.shape

(2098, 3)

In [733]:
df_test.shape

(900, 2)

In [734]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2098 entries, 0 to 2097
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           2098 non-null   int64 
 1   description  2098 non-null   object
 2   object_img   2098 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 49.3+ KB


In [735]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           900 non-null    int64 
 1   description  900 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.2+ KB


## Добавим новых признаков

In [736]:
# длина описания
df_train['num_of_sent'] = df_train['description'].apply(lambda x: len(re.findall('\n',str(x)))+1)
df_test['num_of_sent'] = df_test['description'].apply(lambda x: len(re.findall('\n',str(x)))+1)
# количество слов
df_train['num_of_word'] = df_train['description'].apply(lambda x: len(str(x).split()))
df_test['num_of_word'] = df_test['description'].apply(lambda x: len(str(x).split()))
# количество уникальных слов
df_train['num_of_unique_word'] = df_train['description'].apply(lambda x: len(set(str(x).split())))
df_test['num_of_unique_word'] = df_test['description'].apply(lambda x: len(set(str(x).split())))
# количество символов
df_train['num_of_letters'] = df_train['description'].apply(lambda x: len(str(x)))
df_test['num_of_letters'] = df_test['description'].apply(lambda x: len(str(x)))
# количество знаков препинания
df_train["num_of_punct"] = df_train['description'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test["num_of_punct"] = df_test['description'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
# количество слов капсом
df_train["num_of_words_upper"] = df_train['description'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_test["num_of_words_upper"] = df_test['description'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
# количество слов с заглавной буквы
df_train["num_of_words_title"] = df_train['description'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_test["num_of_words_title"] = df_test['description'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
# количество стоп-слов
df_train["num_of_stopwords"] = df_train['description'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
df_test["num_of_stopwords"] = df_test['description'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
# средняя длина слова в тексте
df_train["mean_word_len"] = df_train['description'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test["mean_word_len"] = df_test['description'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
# процент уникальных слов
df_train['word_unique_percent'] = df_train['num_of_unique_word']*100/df_train['num_of_word']
df_test['word_unique_percent'] = df_test['num_of_unique_word']*100/df_test['num_of_word']
# процент знаков пунктуации
df_train['punct_percent'] = df_train['num_of_punct']*100/df_train['num_of_word']
df_test['punct_percent'] = df_test['num_of_punct']*100/df_test['num_of_word']

In [737]:
df_train['description'][0]

'Фотография. Г. Пермь. Здание горисполкома.  ПОКМ-19594/97 фотобумага,фотопечать     '

In [738]:
df_train['description'][1]

'Фотонегатив пленочный. Труппа театра «У моста» в день презентации театра.  ПОКМ-18907/65 фотопленка,фотосъемка    Селиверстов Михаил Анатольевич '

In [739]:
df_train['description'][1000]

'Фотография. Этюд на Каме. Нижняя Курья. ДИ/Ф-107 ПОКМ-10927/21 фотопечать черно-белая,картон,фотобумага     Художественная фотография выполненная в солнечный и немного облачный [летний] день. Вид на пологий песчаный берег, с краю справа деревянный забор, перед ним тропинка, ниже деревянный тротуар за ним кусты и выход к реке. На переднем плане сосна, на дальнем река, береговая линия и небо.'

In [740]:
df_test['description'][100]

'Фотонегатив стеклянный. Этнография коми-пермяков. Постройка в деревне Ильина Чураковской волости.  ПОКМ-10820/219 стекло     Изображение бревенчатого амбара с полуразрушенной деревянной крышей, невысокой трубой на крыше, открытыми квадратными дверями, перед амбаром насыпь, трава; слева и справа от амбара деревянное ограждение, справа бревна; далее лес.'

Отметим, что можно достать фичи из описания. Для начала можно выделить то, что до первой точки. Потом поработать с серийным номером

In [741]:
df_train['what'] = df_train['description']

In [742]:
df_test['what'] = df_test['description']

In [743]:
def create_what(doc):
    a = ''
    for i in range(len(doc)):
        a = a + doc[i]
        if doc[i] == '.':
            break
    return a

In [744]:
first = df_train.head(1)
first

,id,description,object_img,num_of_sent,num_of_word,num_of_unique_word,num_of_letters,num_of_punct,num_of_words_upper,num_of_words_title,num_of_stopwords,mean_word_len,word_unique_percent,punct_percent,what
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,1,7,7,84,7,2,4,0,10.285714,100.0,100.0,Фотография. Г. Пермь. Здание горисполкома. ПО...


In [745]:
first['what'] = first['description'].apply(create_what)
first

C:\Users\Anastasia\AppData\Local\Temp\ipykernel_11184\689008174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first['what'] = first['description'].apply(create_what)


,id,description,object_img,num_of_sent,num_of_word,num_of_unique_word,num_of_letters,num_of_punct,num_of_words_upper,num_of_words_title,num_of_stopwords,mean_word_len,word_unique_percent,punct_percent,what
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,1,7,7,84,7,2,4,0,10.285714,100.0,100.0,Фотография.


In [746]:
df_train['what'] = df_train['description'].apply(create_what)

In [747]:
df_train

,id,description,object_img,num_of_sent,num_of_word,num_of_unique_word,num_of_letters,num_of_punct,num_of_words_upper,num_of_words_title,num_of_stopwords,mean_word_len,word_unique_percent,punct_percent,what
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,1,7,7,84,7,2,4,0,10.285714,100.000000,100.000000,Фотография.
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,1,15,15,145,5,2,6,1,8.400000,100.000000,33.333333,Фотонегатив пленочный.
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,1,38,37,314,32,5,4,5,7.157895,97.368421,84.210526,"Изразец гладкий расписной ""пермский""- карниз А..."
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526,1,35,34,287,19,3,13,5,7.114286,97.142857,54.285714,Фотонегатив пленочный.
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244,1,43,39,322,13,1,6,10,6.395349,90.697674,30.232558,Фотонегатив стеклянный.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,1464,Фотонегатив пленочный. Сцена из спектакля «Жен...,836,1,17,17,163,9,2,10,2,8.411765,100.000000,52.941176,Фотонегатив пленочный.
2094,1297,Монета ДМ/Н-329 ПОКМ-20597/124 серебро; штампо...,61,1,98,82,622,45,16,13,17,5.336735,83.673469,45.918367,Монета ДМ/Н-329 ПОКМ-20597/124 серебро; штампо...
2095,1507,Фотография. Уральский семинар секретарей комит...,1290,1,15,15,157,6,3,3,2,9.133333,100.000000,40.000000,Фотография.
2096,1863,ПЗС. Подвеска шумящая коньковая на пронизи ПК...,1949,1,71,61,533,27,3,6,10,6.450704,85.915493,38.028169,ПЗС.


In [748]:
df_test['what'] = df_test['description'].apply(create_what)

In [749]:
df_test

,id,description,num_of_sent,num_of_word,num_of_unique_word,num_of_letters,num_of_punct,num_of_words_upper,num_of_words_title,num_of_stopwords,mean_word_len,word_unique_percent,punct_percent,what
0,486,Фотография. Елизавета Алексеевна Юманова. ПКМ...,1,14,14,177,12,1,8,0,11.357143,100.000000,85.714286,Фотография.
1,813,Фотография. Заседание комитета комсомола мотор...,1,61,56,466,24,1,8,14,6.557377,91.803279,39.344262,Фотография.
2,2980,"Фотография. День ""Саланга"". ПОКМ-18530/638 фо...",1,26,25,215,10,1,5,6,7.115385,96.153846,38.461538,Фотография.
3,13,Фотография. Елизавета Алексеевна Юманова. ПКМ...,1,14,14,177,12,1,8,0,11.357143,100.000000,85.714286,Фотография.
4,2467,"Фотография. ""Универмаг Пермь. Оформление демон...",1,19,18,194,10,1,6,2,8.894737,94.736842,52.631579,Фотография.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,2860,Фотонегатив пленочный. Делегаты VI съезда комп...,1,26,26,222,18,5,10,2,7.384615,100.000000,69.230769,Фотонегатив пленочный.
896,2551,"Изразец печной ПОКМ-20318/52 формовка,глина (...",1,5,5,59,5,1,1,0,9.800000,100.000000,100.000000,"Изразец печной ПОКМ-20318/52 формовка,глина (..."
897,2399,"Фотография. ""Универмаг Пермь. Оформление демон...",1,45,42,356,13,1,7,10,6.800000,93.333333,28.888889,Фотография.
898,1492,ПЗС. Подвеска шумящая коньковая ПКМ-21222/274...,1,72,59,548,26,3,5,11,6.555556,81.944444,36.111111,ПЗС.


Отдельно разберём выбросы

In [750]:
first = df_train['description'][0]
first

'Фотография. Г. Пермь. Здание горисполкома.  ПОКМ-19594/97 фотобумага,фотопечать     '

In [751]:
len(first.split(' '))

13

In [752]:
df_train['what'].value_counts(ascending=True).head(30)

Кистень ОР-225 ПОКМ-4777 ковка,чугун,железо,дерево,литье     Гибко-суставное оружие.                                                                                                                                                                                                         1
Кинжал - бебут образца 1907 г.                                                                                                                                                                                                                                                               1
Нагайка  ПОКМ-11453/299 кожа,кость,кустарное производство,дерево     Рукоять из ноги животного с копытцем, с петлей из кожи.                                                                                                                                                                 1
Часы карманные «Павелъ Буре» М-491 ПОКМ-11967/10 эмаль, стекло, металл никелированный, заводское производство диаметр 6,8 см.              

In [753]:
df_test['what'].value_counts(ascending=True).head(30)

Изразец печной  ПОКМ-20318/52 формовка,глина (красная)                                                                                                                                                                                                                                                                       1
Монета ДМ/Н-1237 ПОКМ-3937 серебро; чеканка d - 20 мм 1816 г.                                                                                                                                                                                                                                                                1
Ружье с кремневым  замком ОР-220 ПОКМ-4773 сталь,дерево,кость,производство промышленное     Ствол массивный, восьмигранный с 12-ю нарезами.                                                                                                                                                                                  1
Изразец гладкий расписной - карниз А/РУС-14

In [754]:
def add_what(x):
    if 'Изразец' in x:
        return 'Изразец'
    elif 'Монета' in x:
        return 'Монета'
    elif 'Часы карманные' in x or 'Часы, карманные' in x:
        return 'Часы карманные'
    elif 'Пистолет' in x:
        return 'Пистолет'
    elif 'Шпага' in x:
        return 'Шпага'
    elif 'Пистолет' in x:
        return 'Пистолет'
    elif 'Винтовка' in x:
        return 'Винтовка'
    elif 'Ружье' in x:
        return 'Ружье'
    elif 'Шашка' in x:
        return 'Шашка'
    elif 'Бердыш' in x:
        return 'Бердыш'
    elif 'Кистень' in x:
        return 'Кистень'
    elif 'Штык' in x:
        return 'Штык'
    elif 'Стул' in x:
        return 'Стул'
    elif 'Стрела' in x:
        return 'Стрела'
    elif 'Сабля' in x:
        return 'Сабля'
    elif 'Кокошник' in x:
        return 'Кокошник'
    elif 'Топор' in x:
        return 'Топор'
    elif 'Фотография' in x:
        return 'Фотография'
    elif 'Алебарда' in x:
        return 'Алебарда'
    elif 'Пушка' in x:
        return 'Пушка'
    elif 'Граната' in x:
        return 'Граната'
    elif 'Тесак' in x:
        return 'Тесак'
    elif 'Кинжал' in x:
        return 'Кинжал'
    elif 'Лук' in x:
        return 'Лук'
    elif 'Револьвер' in x:
        return 'Револьвер'
    elif 'Пронизка' in x:
        return 'Пронизка'
    elif 'Копье' in x:
        return 'Копье'
    elif 'Нож' in x:
        return 'Нож'
    elif 'Этюд' in x:
        return 'Этюд'
    elif 'Патрон' in x:
        return 'Патрон'
    elif 'Бомба' in x:
        return 'Бомба'
    elif 'Изображение' in x or 'изображение' in x:
        return 'Изображение'
    elif 'Нагайка' in x:
        return 'Нагайка'
    elif 'Фальконет' in x:
        return 'Фальконет'
    elif 'Ятаган' in x:
        return 'Ятаган'
    elif 'Часы настенные' in x:
        return 'Часы настенные'
    elif 'Часы настольные' in x:
        return 'Часы настольные'
  


    else:
        return x

In [755]:
df_train['what'] = df_train['what'].apply(add_what)

In [756]:
df_test['what'] = df_test['what'].apply(add_what)

In [757]:
df_train

,id,description,object_img,num_of_sent,num_of_word,num_of_unique_word,num_of_letters,num_of_punct,num_of_words_upper,num_of_words_title,num_of_stopwords,mean_word_len,word_unique_percent,punct_percent,what
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,1,7,7,84,7,2,4,0,10.285714,100.000000,100.000000,Фотография
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,1,15,15,145,5,2,6,1,8.400000,100.000000,33.333333,Фотонегатив пленочный.
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,1,38,37,314,32,5,4,5,7.157895,97.368421,84.210526,Изразец
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526,1,35,34,287,19,3,13,5,7.114286,97.142857,54.285714,Фотонегатив пленочный.
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244,1,43,39,322,13,1,6,10,6.395349,90.697674,30.232558,Фотонегатив стеклянный.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,1464,Фотонегатив пленочный. Сцена из спектакля «Жен...,836,1,17,17,163,9,2,10,2,8.411765,100.000000,52.941176,Фотонегатив пленочный.
2094,1297,Монета ДМ/Н-329 ПОКМ-20597/124 серебро; штампо...,61,1,98,82,622,45,16,13,17,5.336735,83.673469,45.918367,Монета
2095,1507,Фотография. Уральский семинар секретарей комит...,1290,1,15,15,157,6,3,3,2,9.133333,100.000000,40.000000,Фотография
2096,1863,ПЗС. Подвеска шумящая коньковая на пронизи ПК...,1949,1,71,61,533,27,3,6,10,6.450704,85.915493,38.028169,ПЗС.


Теперь попробуем вытащить серийный номер

In [758]:
df_train['description'][0]

'Фотография. Г. Пермь. Здание горисполкома.  ПОКМ-19594/97 фотобумага,фотопечать     '

In [759]:
df_train['description'][2]

'Изразец гладкий расписной "пермский"- карниз А/РУС-143 ПОКМ-18870/50 [эмаль (белая, зеленая)],глина (красная),эмалирование,формовка,роспись     Карниз с выпукло-вогнутой лицевой поверхностью и росписью в виде рамки зеленой эмали на белом фоне по краям лицевой пластины (ПГУ 1706/246, Р-2, уч. Г/13, гл.  0,8-1,5 м)'

In [761]:
df_train["len_description"] = df_train.description.map(len)

In [762]:
df_train["object"] = df_train["description"].map(lambda x : x.split()[0]) 

In [763]:
df_test["len_description"] = df_test.description.map(len)

In [764]:
df_test["object"] = df_test["description"].map(lambda x : x.split()[0]) 

In [765]:
df_train["object"] = pd.Categorical(df_train["object"])
df_train["object"].astype('category').cat.codes
df_train["object"] = df_train["object"].cat.codes

In [766]:
df_test["object"] = pd.Categorical(df_test["object"])
df_test["object"].astype('category').cat.codes
df_test["object"] = df_test["object"].cat.codes

In [767]:
df_train.head(3)

,id,description,object_img,num_of_sent,num_of_word,num_of_unique_word,num_of_letters,num_of_punct,num_of_words_upper,num_of_words_title,num_of_stopwords,mean_word_len,word_unique_percent,punct_percent,what,len_description,object
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,1,7,7,84,7,2,4,0,10.285714,100.000000,100.000000,Фотография,84,78
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,1,15,15,145,5,2,6,1,8.400000,100.000000,33.333333,Фотонегатив пленочный.,145,80
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,1,38,37,314,32,5,4,5,7.157895,97.368421,84.210526,Изразец,314,21


Лемматизируем

In [771]:
patterns = "[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
stopwords_en = stopwords.words("english")
stop = stopwords_en+stopwords_ru

morph = pymorphy2.MorphAnalyzer()
def lemmatize(doc):
    doc = doc.lower()
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stop:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
            j = ' '.join(tokens)
    
    return j

In [772]:
df_train['description'] = df_train['description'].apply(lemmatize)

In [773]:
df_test['description'] = df_test['description'].apply(lemmatize)

Сделаем из текста отдельную фичу - посмотрим, что предскажет модель

In [775]:
vectorizer = TfidfVectorizer()
train_X_train = vectorizer.fit_transform(df_train['description'])
test_X_test = vectorizer.transform(df_test['description'])

In [776]:
trans = Normalizer()
X_train_len=trans.fit_transform(df_train['len_description'].values.reshape(-1,1))
X_test_len=trans.transform(df_test['len_description'].values.reshape(-1,1))

In [777]:
train_s=hstack((train_X_train,X_train_len))
test_s=hstack((test_X_test,X_test_len))

In [778]:
y = df_train[['object_img']]

In [779]:
params_lgb = {
    "n_estimators": 1000,
    "verbose": -1
}

In [780]:
model = LGBMRegressor(**params_lgb)
model.fit(train_s, y)

C:\Users\Anastasia\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMRegressor(n_estimators=1000, verbose=-1)

In [781]:
df_train['text_first_pred'] = model.predict(train_s)

C:\Users\Anastasia\anaconda3\lib\site-packages\lightgbm\basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [782]:
df_test['text_first_pred'] = model.predict(test_s)

C:\Users\Anastasia\anaconda3\lib\site-packages\lightgbm\basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


## Выделим выборки

In [783]:
X = df_train.drop(["object_img", 'description'], axis = 1)
X_test = df_test.drop('description', axis=1)
y = df_train['object_img']

In [784]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2098 entries, 0 to 2097
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   2098 non-null   int64  
 1   num_of_sent          2098 non-null   int64  
 2   num_of_word          2098 non-null   int64  
 3   num_of_unique_word   2098 non-null   int64  
 4   num_of_letters       2098 non-null   int64  
 5   num_of_punct         2098 non-null   int64  
 6   num_of_words_upper   2098 non-null   int64  
 7   num_of_words_title   2098 non-null   int64  
 8   num_of_stopwords     2098 non-null   int64  
 9   mean_word_len        2098 non-null   float64
 10  word_unique_percent  2098 non-null   float64
 11  punct_percent        2098 non-null   float64
 12  what                 2098 non-null   object 
 13  len_description      2098 non-null   int64  
 14  object               2098 non-null   int8   
 15  text_first_pred      2098 non-null   f

In [785]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   900 non-null    int64  
 1   num_of_sent          900 non-null    int64  
 2   num_of_word          900 non-null    int64  
 3   num_of_unique_word   900 non-null    int64  
 4   num_of_letters       900 non-null    int64  
 5   num_of_punct         900 non-null    int64  
 6   num_of_words_upper   900 non-null    int64  
 7   num_of_words_title   900 non-null    int64  
 8   num_of_stopwords     900 non-null    int64  
 9   mean_word_len        900 non-null    float64
 10  word_unique_percent  900 non-null    float64
 11  punct_percent        900 non-null    float64
 12  what                 900 non-null    object 
 13  len_description      900 non-null    int64  
 14  object               900 non-null    int8   
 15  text_first_pred      900 non-null    flo

In [786]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## Работа с моделью 

Возьмём модель CatBoost, поскольку она умеет работать с категориальными фичами

In [792]:
cat_features = ['what']

In [793]:
cat_model = CatBoostRegressor(loss_function='MAE', verbose=10, cat_features=cat_features)

Далее идёт подбор гиперпараметров (этот код выполнился 1 раз и далее я его закомментировала)

In [794]:
#parameters = { 
#              'depth':[8, 10, 12],
#              'iterations':[1001, 1501, 2001, 2501]
#              }
#model = GridSearchCV(cat_model, parameters, cv=3).fit(X_train, y_train)
#print('Best score and parameter combination:')
#print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.9972594596091668 {'depth': 8, 'iterations': 2501}

In [795]:
#parameters = { 
#              'depth':[4, 5, 6],
#              'iterations':[2601,2701]
#              }
#model = GridSearchCV(cat_model, parameters, cv=2).fit(X_train, y_train)
#print('Best score and parameter combination:')
#print(model.best_score_, model.best_params_)

0.9983140752612604 {'depth': 6, 'iterations': 2601}

Обучим модель

In [796]:
cat_model = CatBoostRegressor(loss_function='MAE', verbose=10, depth=6, iterations=2601, cat_features=cat_features)
cat_model.fit(X, y)

0:	learn: 730.3023398	total: 17ms	remaining: 44.1s
10:	learn: 533.8282954	total: 235ms	remaining: 55.3s
20:	learn: 397.5822273	total: 495ms	remaining: 1m
30:	learn: 298.2190925	total: 690ms	remaining: 57.2s
40:	learn: 226.4839322	total: 882ms	remaining: 55.1s
50:	learn: 173.8378727	total: 1.12s	remaining: 55.9s
60:	learn: 137.5422287	total: 1.38s	remaining: 57.4s
70:	learn: 110.5170063	total: 1.62s	remaining: 57.9s
80:	learn: 89.2193008	total: 1.86s	remaining: 57.8s
90:	learn: 74.5841983	total: 2.09s	remaining: 57.7s
100:	learn: 63.6232001	total: 2.3s	remaining: 57s
110:	learn: 55.7505682	total: 2.53s	remaining: 56.8s
120:	learn: 49.7344270	total: 2.8s	remaining: 57.4s
130:	learn: 44.6282666	total: 3.01s	remaining: 56.8s
140:	learn: 40.6190542	total: 3.22s	remaining: 56.2s
150:	learn: 37.3270713	total: 3.43s	remaining: 55.7s
160:	learn: 34.6447455	total: 3.63s	remaining: 55s
170:	learn: 32.9378345	total: 3.83s	remaining: 54.5s
180:	learn: 31.3945135	total: 4.07s	remaining: 54.4s
190:	l

Посмотрим, какие фичи наиолее важными оказались

In [797]:
cat_model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,text_first_pred,83.469680
1,object,3.042710
2,len_description,2.217452
3,num_of_unique_word,1.822656
4,num_of_letters,1.633174
5,num_of_punct,1.341006
6,word_unique_percent,1.267045
7,num_of_stopwords,1.089894
8,num_of_words_title,0.984274
9,num_of_words_upper,0.869210


## Оценка точности

In [798]:
pred = cat_model.predict(X_test)

In [800]:
df_sub = pd.read_csv('sample_solution.csv')

In [801]:
df_sub

,id,object_img
0,486,0
1,813,0
2,2980,0
3,13,0
4,2467,0
...,...,...
895,2860,0
896,2551,0
897,2399,0
898,1492,0


In [802]:
df_sub["object_img"] = pred.round()
df_sub.head(4)

,id,object_img
0,486,1212.0
1,813,1588.0
2,2980,1371.0
3,13,1201.0


Посмотрим, что модель напредсказывала

In [803]:
df_sub['object_img'].value_counts()

2719.0    4
1053.0    3
882.0     3
1023.0    3
1384.0    3
         ..
1728.0    1
2673.0    1
2278.0    1
1690.0    1
1629.0    1
Name: object_img, Length: 753, dtype: int64

Данные не должны повторяться, а кроме того не должны повторяться с тестовым датасетом, поэтому подкорректируем предсказания

Соберём в список предсказания модели

In [804]:
list_sub = list(df_sub['object_img'].astype(int))

In [805]:
list_sub

[1212,
 1588,
 1371,
 1201,
 522,
 1644,
 65,
 1288,
 2356,
 2532,
 156,
 1710,
 2732,
 2753,
 1956,
 545,
 1670,
 1797,
 678,
 1636,
 1023,
 1633,
 2229,
 2778,
 2749,
 2300,
 2839,
 549,
 2757,
 2187,
 1133,
 48,
 1139,
 2881,
 1401,
 778,
 688,
 455,
 778,
 1150,
 734,
 2320,
 1093,
 64,
 2857,
 2024,
 1308,
 1099,
 722,
 2555,
 2259,
 777,
 1980,
 2356,
 2739,
 2348,
 2719,
 726,
 2825,
 2458,
 551,
 89,
 2791,
 1308,
 782,
 599,
 104,
 2521,
 675,
 2482,
 1509,
 685,
 2574,
 2155,
 754,
 1832,
 170,
 931,
 2562,
 531,
 1343,
 135,
 2604,
 677,
 2714,
 524,
 505,
 657,
 760,
 244,
 2574,
 2435,
 817,
 2600,
 2042,
 1290,
 1718,
 387,
 1370,
 2262,
 177,
 2963,
 2306,
 788,
 2558,
 2286,
 2120,
 1789,
 1040,
 2349,
 2322,
 1095,
 1506,
 2602,
 2277,
 984,
 2357,
 2441,
 2011,
 1204,
 779,
 2239,
 653,
 2268,
 2598,
 283,
 1153,
 2765,
 1332,
 290,
 1363,
 1399,
 2053,
 1582,
 163,
 637,
 2885,
 806,
 2513,
 1895,
 2902,
 2111,
 494,
 948,
 2388,
 2172,
 465,
 1256,
 2717,
 1256,
 71

Составим множество всех возможных ответов и для train и для test

In [806]:
set_list = list(range(1, 2999, 1))

In [807]:
set_list

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

Составим отдельный список для тех изображений, которые заняты train-ом

In [808]:
train_s = list(df_train['object_img'])
train_s

[799,
 854,
 1794,
 526,
 244,
 917,
 1069,
 2114,
 2555,
 946,
 1093,
 474,
 2681,
 64,
 745,
 2795,
 2631,
 2919,
 1163,
 2815,
 650,
 1515,
 1983,
 1570,
 1870,
 1009,
 2467,
 2446,
 1626,
 758,
 2255,
 570,
 2741,
 207,
 2176,
 1444,
 1317,
 695,
 609,
 2464,
 1448,
 1238,
 2977,
 1351,
 129,
 1074,
 1314,
 1364,
 44,
 67,
 2846,
 1477,
 2605,
 2533,
 746,
 2664,
 655,
 287,
 701,
 1050,
 806,
 720,
 1169,
 234,
 161,
 208,
 1455,
 1836,
 2863,
 676,
 125,
 400,
 2138,
 1019,
 1298,
 1140,
 1503,
 2936,
 1323,
 1555,
 617,
 2262,
 1184,
 1952,
 1996,
 891,
 2419,
 372,
 2641,
 2013,
 1490,
 2627,
 2069,
 76,
 922,
 2462,
 2762,
 1823,
 2527,
 2614,
 1616,
 7,
 2864,
 2441,
 1385,
 2928,
 1031,
 2703,
 2369,
 409,
 63,
 1730,
 2727,
 1571,
 2084,
 288,
 319,
 514,
 909,
 804,
 1382,
 220,
 1634,
 2213,
 1752,
 1670,
 2025,
 558,
 803,
 1445,
 2225,
 467,
 1360,
 2773,
 858,
 1759,
 2452,
 2399,
 672,
 304,
 236,
 2894,
 2398,
 2120,
 1466,
 1778,
 2348,
 1910,
 2736,
 263,
 2006,
 1

Положим в отдельный список множество всех ответов, которые возможны для тестовых данных. Это соответственно те элементы, которые встречаются в списке всех возможных ответов и при этом не заняты train-ом

In [809]:
set_list_f = [x for x in set_list if x not in train_s]
set_list_f

[8,
 9,
 11,
 13,
 14,
 18,
 23,
 25,
 26,
 27,
 28,
 31,
 35,
 43,
 47,
 50,
 51,
 55,
 66,
 82,
 84,
 85,
 92,
 95,
 96,
 99,
 101,
 102,
 107,
 109,
 112,
 116,
 119,
 126,
 130,
 136,
 144,
 162,
 164,
 168,
 174,
 181,
 184,
 186,
 189,
 193,
 204,
 211,
 212,
 216,
 227,
 228,
 233,
 239,
 247,
 252,
 253,
 254,
 256,
 257,
 260,
 261,
 265,
 266,
 273,
 275,
 279,
 283,
 291,
 292,
 295,
 300,
 302,
 306,
 307,
 313,
 316,
 320,
 323,
 324,
 331,
 333,
 341,
 343,
 346,
 348,
 350,
 351,
 353,
 355,
 358,
 362,
 364,
 365,
 366,
 369,
 370,
 375,
 377,
 380,
 382,
 385,
 386,
 390,
 391,
 395,
 404,
 407,
 421,
 422,
 423,
 424,
 425,
 427,
 428,
 429,
 438,
 439,
 440,
 441,
 444,
 450,
 451,
 452,
 455,
 456,
 458,
 464,
 479,
 484,
 487,
 493,
 494,
 495,
 499,
 504,
 506,
 508,
 511,
 512,
 521,
 525,
 528,
 533,
 539,
 542,
 544,
 550,
 553,
 568,
 571,
 572,
 575,
 576,
 577,
 582,
 587,
 588,
 594,
 595,
 598,
 604,
 606,
 608,
 611,
 615,
 616,
 618,
 623,
 633,
 634,
 6

Проверим, что длины списков совпадают

In [810]:
len(list_sub)

900

In [811]:
len(set_list_f)

900

In [818]:
set_list_f

[8,
 9,
 11,
 13,
 14,
 18,
 23,
 25,
 26,
 27,
 28,
 31,
 35,
 43,
 47,
 50,
 51,
 55,
 66,
 82,
 84,
 85,
 92,
 95,
 96,
 99,
 101,
 102,
 107,
 109,
 112,
 116,
 119,
 126,
 130,
 136,
 144,
 162,
 164,
 168,
 174,
 181,
 184,
 186,
 189,
 193,
 204,
 211,
 212,
 216,
 227,
 228,
 233,
 239,
 247,
 252,
 253,
 254,
 256,
 257,
 260,
 261,
 265,
 266,
 273,
 275,
 279,
 283,
 291,
 292,
 295,
 300,
 302,
 306,
 307,
 313,
 316,
 320,
 323,
 324,
 331,
 333,
 341,
 343,
 346,
 348,
 350,
 351,
 353,
 355,
 358,
 362,
 364,
 365,
 366,
 369,
 370,
 375,
 377,
 380,
 382,
 385,
 386,
 390,
 391,
 395,
 404,
 407,
 421,
 422,
 423,
 424,
 425,
 427,
 428,
 429,
 438,
 439,
 440,
 441,
 444,
 450,
 451,
 452,
 455,
 456,
 458,
 464,
 479,
 484,
 487,
 493,
 494,
 495,
 499,
 504,
 506,
 508,
 511,
 512,
 521,
 525,
 528,
 533,
 539,
 542,
 544,
 550,
 553,
 568,
 571,
 572,
 575,
 576,
 577,
 582,
 587,
 588,
 594,
 595,
 598,
 604,
 606,
 608,
 611,
 615,
 616,
 618,
 623,
 633,
 634,
 6

In [819]:
X_test['pred'] = pred

In [820]:
X_test

,id,num_of_sent,num_of_word,num_of_unique_word,num_of_letters,num_of_punct,num_of_words_upper,num_of_words_title,num_of_stopwords,mean_word_len,word_unique_percent,punct_percent,what,len_description,object,text_first_pred,pred
0,486,1,14,14,177,12,1,8,0,11.357143,100.000000,85.714286,Фотография,177,52,1218.445483,1211.751203
1,813,1,61,56,466,24,1,8,14,6.557377,91.803279,39.344262,Фотография,466,52,1541.950481,1587.818863
2,2980,1,26,25,215,10,1,5,6,7.115385,96.153846,38.461538,Фотография,215,52,1410.625811,1371.188126
3,13,1,14,14,177,12,1,8,0,11.357143,100.000000,85.714286,Фотография,177,52,1220.082967,1200.956961
4,2467,1,19,18,194,10,1,6,2,8.894737,94.736842,52.631579,Фотография,194,52,577.381444,522.446737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,2860,1,26,26,222,18,5,10,2,7.384615,100.000000,69.230769,Фотонегатив пленочный.,222,56,547.217192,542.765555
896,2551,1,5,5,59,5,1,1,0,9.800000,100.000000,100.000000,Изразец,59,13,1872.502972,1885.769576
897,2399,1,45,42,356,13,1,7,10,6.800000,93.333333,28.888889,Фотография,356,52,902.140644,762.526554
898,1492,1,72,59,548,26,3,5,11,6.555556,81.944444,36.111111,ПЗС.,548,31,1897.737150,1888.225378


Добавим колонку, в которой будем сохранять финальный результат

In [822]:
X_test['final'] = X_test['pred'].copy()

In [823]:
set_list_f = [x for x in set_list if x not in train_s]
len(set_list_f)

900

Пройдёмся переменной i по списку возможных значений для теста. Будем смотреть, чтобы значение, которое мы рассматриваем находилось в пормежутке между значениями предсказанными catboost и lgbm +- определённое значение k (увеличивающиеся с каждым шагом). В случае удовлетворения условию, значение в колонке final будет равно этому значению i. Кроме того, нужно не забыть убрать это значение из списка. Таким образом, мы обеспечим выполнения условий о том, что значения должны:
1. Не повторяться
2. Не включать в себя индексы изображений из папки train


In [824]:
for k in range(0, 5000):
    for j in range(len(X_test['pred'])):
        if X_test['pred'][j] == X_test['final'][j]:
            for i in range(len(set_list_f)):
                if ((set_list_f[i] in range(abs(round(X_test['pred'][j])-k), abs(round(abs(X_test['text_first_pred'][j])))+k))
                or (set_list_f[i] in range(abs(round(X_test['text_first_pred'][j])-k), abs(round(X_test['pred'][j])+k)))):
                 
                    X_test['final'][j] = set_list_f[i]
                    set_list_f.pop(i)
                    i-=1
                    break

C:\Users\Anastasia\AppData\Local\Temp\ipykernel_11184\404069021.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['final'][j] = set_list_f[i]


Проверим, что все возможные элемента списка для test-а "разобраны"

In [825]:
len(set_list_f)

0

In [826]:
X_test

,id,num_of_sent,num_of_word,num_of_unique_word,num_of_letters,num_of_punct,num_of_words_upper,num_of_words_title,num_of_stopwords,mean_word_len,word_unique_percent,punct_percent,what,len_description,object,text_first_pred,pred,final
0,486,1,14,14,177,12,1,8,0,11.357143,100.000000,85.714286,Фотография,177,52,1218.445483,1211.751203,1213.0
1,813,1,61,56,466,24,1,8,14,6.557377,91.803279,39.344262,Фотография,466,52,1541.950481,1587.818863,1546.0
2,2980,1,26,25,215,10,1,5,6,7.115385,96.153846,38.461538,Фотография,215,52,1410.625811,1371.188126,1372.0
3,13,1,14,14,177,12,1,8,0,11.357143,100.000000,85.714286,Фотография,177,52,1220.082967,1200.956961,1202.0
4,2467,1,19,18,194,10,1,6,2,8.894737,94.736842,52.631579,Фотография,194,52,577.381444,522.446737,525.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,2860,1,26,26,222,18,5,10,2,7.384615,100.000000,69.230769,Фотонегатив пленочный.,222,56,547.217192,542.765555,452.0
896,2551,1,5,5,59,5,1,1,0,9.800000,100.000000,100.000000,Изразец,59,13,1872.502972,1885.769576,1879.0
897,2399,1,45,42,356,13,1,7,10,6.800000,93.333333,28.888889,Фотография,356,52,902.140644,762.526554,906.0
898,1492,1,72,59,548,26,3,5,11,6.555556,81.944444,36.111111,ПЗС.,548,31,1897.737150,1888.225378,1888.0


Убедимся, что значения не повторяются

In [827]:
X_test['final'].value_counts()

1213.0    1
239.0     1
1518.0    1
1485.0    1
2922.0    1
         ..
1149.0    1
1787.0    1
901.0     1
864.0     1
1627.0    1
Name: final, Length: 900, dtype: int64

Подготовим ответ

In [828]:
df_sub["object_img"] =  X_test['final']

In [829]:
df_sub.to_csv("submission.csv", index = False)